In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df_in = pd.read_json(path+filename, lines=True)

In [3]:
#Divido il dataframe per intervalli settimanali, riduco le colonne a quelle che mi initeressano
df_in['time_utc'] = pd.to_datetime(df_in['utc_datetime_str'])
df_small = df_in[['time_utc','GPT_label','polarity','text']].copy()
week_intervals = pd.date_range(start=df_small['time_utc'].min(), end=df_small['time_utc'].max(), freq='7D')

week_df_list = []
for i in range(len(week_intervals) - 1):
    start_date = week_intervals[i]
    end_date = week_intervals[i + 1]
    week_data = df_small[(df_small['time_utc'] >= start_date) & (df_small['time_utc'] < end_date)].copy()
    week_df_list.append(week_data)
week_df_list.append(df_small[(df_small['time_utc'] >= end_date) & (df_small['time_utc'] <= df_small['time_utc'].max())].copy())

In [4]:
topics = list(df_small['GPT_label'].unique())

In [5]:
#ottengo un dizionario in cui le chiavi sono i topic e i valori sono liste di DFs. Ogni lista contiene i DFs per quei topic divisi per settimana
dfs_per_weeks_and_topics = {}
for topic in topics:
    week_df_list_per_topic = []
    for weekly_df in week_df_list:
        week_df_list_per_topic.append(weekly_df[weekly_df['GPT_label'] == topic])
    dfs_per_weeks_and_topics[topic] = week_df_list_per_topic

In [6]:
week_polarity_avg_t = {}
week_pos_polarity_avg_t = {}
week_neg_polarity_avg_t = {}

In [7]:
for topic in topics:
    week_polarity_avg_t[topic] = []
    week_pos_polarity_avg_t[topic] = []
    week_neg_polarity_avg_t[topic] = []
    for df in dfs_per_weeks_and_topics[topic]:
        week_polarity_avg_t[topic].append(df['polarity'].mean())
        week_pos_polarity_avg_t[topic].append(df['polarity'][df['polarity'] > 0].mean())
        week_neg_polarity_avg_t[topic].append(df['polarity'][df['polarity'] < 0].mean())

In [8]:
volume_post_per_week_t = {}
for topic in topics:
    volume_post_per_week_t[topic] = []
    for topic_weekly_df in dfs_per_weeks_and_topics[topic]:
        volume_post_per_week_t[topic].append(topic_weekly_df.shape[0])

In [21]:
def plot(avg_list, avg_pos_list, avg_neg_list, volume_list, topic):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.02, 
                    row_heights=[0.2, 0.7])
    fig.add_trace(go.Scatter(x=week_intervals, y=avg_list, mode='lines', name='Polarity Total'), row=2, col=1)
    fig.add_trace(go.Scatter(x=week_intervals, y=avg_pos_list, mode='lines', name='Polarity Positive'), row=2, col=1)
    fig.add_trace(go.Scatter(x=week_intervals, y=avg_neg_list, mode='lines', name='Polarity Negative'), row=2, col=1)

    fig.add_trace(go.Scatter(x=week_intervals, y=volume_list, name='Data Volume'), row=1, col=1)

    fig.update_layout(title=f'Polarity Analysis Over Time with Data Volume topic: {topic}',
                  xaxis_title='Date',
                  yaxis_title='Volume',
                  yaxis2_title='Polarity',
                  showlegend=True)

    fig.show()


In [22]:
for topic in topics:   
    plot(week_polarity_avg_t[topic], week_pos_polarity_avg_t[topic], week_neg_polarity_avg_t[topic], volume_post_per_week_t[topic], topic)
